In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import os
import re
import numpy as np
from collections import defaultdict
from math import log

In [28]:
# Preprocessing function
def preprocess(text):
    return re.findall(r'\b\w+\b', text.lower())

In [29]:
# Load documents
def load_documents(folder_path):
    docs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                docs[filename] = preprocess(file.read())
    return docs

In [30]:
# Define the queries manually (for testing purpose)
queries = [
       "battery life out",
       "qualifying amount",
       "international purchases"
    ]

In [31]:
# Compute term frequencies and document frequencies
def compute_statistics(docs):
    doc_count = len(docs)
    term_doc_freq = defaultdict(int)
    term_freq = defaultdict(lambda: defaultdict(int))

    for doc_id, words in docs.items():
        word_set = set(words)
        for word in words:
            term_freq[doc_id][word] += 1
        for word in word_set:
            term_doc_freq[word] += 1

    return term_freq, term_doc_freq, doc_count

In [32]:
# Compute relevance probabilities using BIM
def compute_relevance_prob(query, term_freq, term_doc_freq, doc_count):
    scores = {}
    for doc_id in term_freq:
        score = 1.0
        for term in query:
            tf = term_freq[doc_id].get(term, 0)
            df = term_doc_freq.get(term, 0)
            p_term_given_relevant = (tf + 1) / (sum(term_freq[doc_id].values()) + len(term_doc_freq))
            p_term_given_not_relevant = (df + 1) / (doc_count - df + len(term_doc_freq))
            score *= (p_term_given_relevant / p_term_given_not_relevant)
        scores[doc_id] = score
    return scores

In [34]:
# Main retrieval function
def retrieve_documents(folder_path, queries):
    docs = load_documents(folder_path)
    queries = [
       "battery life out",
       "qualifying amount",
       "international purchases"
    ]

    term_freq, term_doc_freq, doc_count = compute_statistics(docs)

    # Open a file to write results
    with open('Pawan_result.txt', 'w') as result_file:
        for query in queries:
            query_terms = preprocess(query)
            scores = compute_relevance_prob(query_terms, term_freq, term_doc_freq, doc_count)
            ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)

            # Print to console
            print(f"Query: {query}")
            result_file.write(f"Query: {query}\n")

            # Print and write only the top 3 results with ranks
            for rank, (doc_id, score) in enumerate(ranked_docs[:3], start=1):
                print(f"Rank {rank}: Document:{doc_id}, Score: {score:.4f}")
                result_file.write(f"Rank {rank}: Document {doc_id}, Score: {score:.4f}\n")

            print()
            result_file.write("\n")

# Example usage
folder_path = '/content/drive/MyDrive/Dataset: IR'
retrieve_documents(folder_path, queries)

Query: battery life out
Rank 1: Document:doc.1.txt, Score: 0.0697
Rank 2: Document:doc.7.txt, Score: 0.0371
Rank 3: Document:docs_11.txt, Score: 0.0299

Query: qualifying amount
Rank 1: Document:doc.7.txt, Score: 1.0663
Rank 2: Document:doc.5.txt, Score: 0.2325
Rank 3: Document:Doc.10.txt, Score: 0.1402

Query: international purchases
Rank 1: Document:Doc.9.txt, Score: 1.2607
Rank 2: Document:Doc.10.txt, Score: 0.2106
Rank 3: Document:docs_20.txt, Score: 0.1864

